<a href="https://www.kaggle.com/code/tarizatique/web-scraping-and-sentiment-anaysis-imdb?scriptVersionId=106053539" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Objectives of Project

1. Reviews Preprocessing and Cleaning
2. Story Generation and Visualization from reviews
3. Extracting Features from Cleaned reviews
4. Building Wordclouds
5. Sentiment Analysis

In [ ]:

import pandas as pd
import requests
import pandas as pd
from bs4 import BeautifulSoup
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('omw-1.4')
from nltk.util import ngrams
from wordcloud import WordCloud, STOPWORDS
nltk.download('punkt')
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
#pasting links of the website we want to scrap
url = "https://www.imdb.com/title/tt0111161/reviews?ref_=tt_urv"
page = requests.get(url)
page


In [ ]:
#changing the raw format to  alignment
soup = BeautifulSoup(page.content,"html.parser")
soup.prettify()
scrape_review = soup.find_all('div', class_="text show-more__control")
#running these may take too much space and it will waste enough time in scrolling


In [ ]:
#creating an empty list naming movie and adding all review into them
#sparse review
movies = []
for movie in scrape_review:
    movie = movie.get_text().replace('\n', "")
    movie = movie.strip(" ")
    movies.append(movie)
    movie = movie.strip(" ")
   

In [ ]:
#you can chekc all review here
movies

**Text Cleaning**

In [ ]:
# writng reviews in a text file 
with open("movies.txt", "w", encoding='utf8') as output:
    output.write(str(movies))
    
    # Joinining all the reviews into single paragraph 
    ip_rev_string = " ".join(movies)

In [ ]:


# Removing unwanted symbols incase if exists
ip_rev_string = re.sub("[^A-Za-z" "]+", " ", ip_rev_string).lower()
# ip_rev_string = re.sub("[0-9" "]+"," ", ip_rev_string)

# words that contained in the reviews
ip_reviews_words = ip_rev_string.split(" ")

ip_reviews_words = ip_reviews_words[1:]

In [ ]:
#converting into sparse matrix
cv = CountVectorizer()
X =cv.fit_transform(ip_reviews_words)

In [ ]:
#adding my stopwords custom gile you can either download it from nltk library
with open("../input/stopwords/stopwords.txt", "r") as sw:
    stop_words = sw.read()
    
stop_words = stop_words.split("\n")
#adding some custom that are not present in stopwords file
stop_words.extend(["movie","film","shawshank","time","prison","tim robbins","morgan","freeman","stephen","king","tim","robbins","andy","shawsank redemption","redemption","character","movies","tim robbins","stephen king","morgan freeman","frank darabont","andy dufresne"])

ip_reviews_words = [w for w in ip_reviews_words if not w in stop_words]

# Joinining all the reviews into single paragraph 
ip_rev_string = " ".join(ip_reviews_words)

In [ ]:
# WordCloud can be performed on the string inputs.
# Corpus level word cloud

wordcloud_ip = WordCloud(background_color='White',
                      width=1800,
                      height=1400
                     ).generate(ip_rev_string)
plt.imshow(wordcloud_ip)

In [ ]:
# positive words # Choose the path for +ve words stored in system
with open("../input/positvie-words/positve words.txt","r") as pos:
  poswords = pos.read().split("\n")

# Positive word cloud
# Choosing the only words which are present in positive words
ip_pos_in_pos = " ".join ([w for w in ip_reviews_words if w in poswords])

In [ ]:
wordcloud_pos_in_pos = WordCloud(
                      background_color='White',
                      width=1800,
                      height=1400
                     ).generate(ip_pos_in_pos)
plt.figure(2)
plt.imshow(wordcloud_pos_in_pos)

In [ ]:
# negative words Choose path for -ve words stored in system
with open("../input/stopwords/Negative words.txt", "r") as neg:
  negwords = neg.read().split("\n")

# negative word cloud
# Choosing the only words which are present in negwords
ip_neg_in_neg = " ".join ([w for w in ip_reviews_words if w in negwords])

In [ ]:
#wordcloud with negative words
wordcloud_neg_in_neg = WordCloud(
                      background_color='black',
                      width=1800,
                      height=1400
                     ).generate(ip_neg_in_neg)
plt.figure(3)
plt.imshow(wordcloud_neg_in_neg)

In [ ]:
#lemmetization
WNL = nltk.WordNetLemmatizer()

In [ ]:
# Lowercase and tokenize
text = ip_rev_string.lower()

# Remove single quote early since it causes problems with the tokenizer.
text = text.replace("'", "")

tokens = nltk.word_tokenize(text)
text1 = nltk.Text(tokens)


In [ ]:

# Remove extra chars and remove stop words.
text_content = [''.join(re.split("[ .,;:!?‘’``''@#$%^_&*()<>{}~\n\t\\\-]", word)) for word in text1]

# Create a set of stopwords
stopwords_wc = set(STOPWORDS)
customised_words = ["movie","film","shawshank","time","prison","tim robbins","morgan","freeman","stephen","king","tim","robbins","andy","shawsank redemption","redemption","character","movies","tim robbins","stephen king","morgan freeman","frank darabont","andy dufresne" ]
# If you want to remove any particular word form text which does not contribute much in meaning

new_stopwords = stopwords_wc.union(customised_words)


In [ ]:
# Remove stop words
text_content = [word for word in text_content if word not in new_stopwords]

# Take only non-empty entries
text_content = [s for s in text_content if len(s) != 0]

# Best to get the lemmas of each word to reduce the number of similar words
text_content = [WNL.lemmatize(t) for t in text_content]
 

In [ ]:
#Unigram
unigram = []
n = 1
unigrams = ngrams(text_content, n)
  for item in unigrams:
   
   print(item)

**Bigrams**

In [ ]:

bigrams_list = list(nltk.bigrams(text_content))
# we can show and print ot
print(bigrams_list)

In [ ]:
dictionary2 = [' '.join(tup) for tup in bigrams_list]
print (dictionary2)

In [ ]:
# Using count vectoriser to view the frequency of bigrams
vectorizer = CountVectorizer(ngram_range=(2, 2))
bag_of_words = vectorizer.fit_transform(dictionary2)
vectorizer.vocabulary_

In [ ]:
sum_words = bag_of_words.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
print(words_freq[:100])

In [ ]:
# Generating wordcloud
words_dict = dict(words_freq)
WC_height = 1000
WC_width = 1500
WC_max_words = 100
wordCloud = WordCloud(max_words=WC_max_words, height=WC_height, width=WC_width, stopwords=new_stopwords)

wordCloud.generate_from_frequencies(words_dict)
plt.figure(4)
plt.title('Most frequently occurring bigrams connected by same colour and font size')
plt.imshow(wordCloud, interpolation='bilinear')
plt.axis("off")
plt.show()